In [ ]:
%%sh
python3 manage.py runserver 9999

# Django ORM 

Object Relational Mapping. `django.db` `models` provides class interface to a database. Model definition is converted into database queries and object mappings by django. *SqlAlchemy* is another popular tool. Django has its own implementation.

`models.py` is expected to contain your data description. Any class derived from `models.Model` will be a relational model.

```python
from django.db import models

class Student(models.Model):
	sid = models.CharField(max_length=10, primary_key = True)
	name = models.CharField(max_length=30)
	surname = models.CharField(max_length=30)
    # if _str_ is implemented object details are displayed in query results
    def __str__(self):
		return ' '.join([str(self.sid),str(self.name),str(self.surname)])
```

This definition will generate all necessary queries to generate and access a row called `<app>_<class>`:

```SQL
CREATE TABLE "student_student" (
"sid" varchar(10) NOT NULL PRIMARY KEY, 
"name" varchar(30) NOT NULL, 
"surname" varchar(30) NOT NULL);
```

Some important field types are:
`AutoField, BinaryField, BooleanField, CharField, DecimalField, DateTimeField, DateField, TimeField, EmailField, FileField, FloatField, ImageField, IntegerField, GenericIPAddressField, URLField`

Fields get some options:
 * `null` (bool) if field can be null.
 * `primary_key` (bool) if this field is the primary key. If no primary key specified an autogenerated key field is added (called `id`)
 * `db_index` (bool) if index should be created in database. Secondary indexes are created by this flag.
 * `default` a default value if field is unspecified.
 * `unique` (bool)
 * `max_length` for string like fields, maximum size allowed
 * `auto_now` for date and time fields if current date/time is automatically set.


In [8]:
import os
import sys

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "d2017.settings")
import django
django.setup()

from student.models import Department, Student, Course

## Adding objects in database

simply constructing an object from class with parameters. All fields can be assigned in the object later. `save()` method creates the object (row in the table)

In [9]:
st = Student(55727, 'Onur Tolga', 'Şehitoğlu')
st.save()
Student(name='Ali', surname='Cin', sid=211112).save()
st.sid = 44444
# since primary key changed, save will generate another object
st.save()
# now query all objects
Student.objects.all()

<QuerySet [<Student: 211112 Ali Cin>, <Student: 44444 Onur Tolga Şehitoğlu>, <Student: 123415 Bugs Bunny>, <Student: 423122 Daffy Duck>, <Student: 314213 Sylvester Cat>, <Student: 423145 Tweety Tweets>, <Student: 221412 Tazmanian Devil>, <Student: 23523523 weqwe qweqwe>, <Student: 324243 asdasd  lczxc zxzxc>, <Student: 55727 Onur Tolga Şehitoğlu>]>

Add more objects. You can use `objects.create()` method of class to create and save in a single call.

In [10]:
for sid,name,sname in [(123415,'Bugs','Bunny'),(423122,'Daffy','Duck'),(314213,'Sylvester','Cat'),(423145, 'Tweety','Tweets'),
      (76341,'Marty','Martian'),(221412,'Tasmanian','Devil'),(652141,'Elmor','Fudd')]:
    Student.objects.create(sid=sid,name=name,surname=sname)

IntegrityError: UNIQUE constraint failed: student_student.sid

## Queries

* `<classname>.objects` give a query interface with various methods. Some methods return a `QuerySet` object that can be iterated or accessed by an index (like a `list`)
* `all()` method returns all objects (`select * from <table>`)
* `get(<lookup>[,<lookup>]*)` method returns only one object. Query result should be unique
* `filter(<loookup>[,<lookup>]*)` method returns all objects matching lookup (`select ... where...`)
* some Field lookups are:
  - `fieldname = value` or `fieldname__exact = value` . Equality  (`select .... where fieldname=value`)
  - `fieldname__contains = value` string contains the value (`select ... where fieldname like '%value%'`)
  - `fieldname__gt = value` greater than (`select ... where fieldname > value`). Other comparison operators: `gte, lt, lte`
  - `fieldname__startswith = value` string starts with the value (`select ... where fieldname like 'value%'). Also `endswith` works same way.
  - `fieldname__in = iteratable` if field name in the following set of values that are iterated. `fieldname_in = ['a','b','c']` (`select ... where fieldname IN ('a','b','c')`). Also other query result can be used as:
     ```python
        cset = Department.objects.filter(did__ge='571')
        st = Student.objects.filter(department__in = cset)
     ```
  - `fieldname__range = (start, end)` range test, integer, string or date/time fields. (`SELECT ... WHERE fieldname BETWEEN (start,end)`. Range is inclusive.
  - `fieldname__date=value`,  matches date part of the date or datetime field. Similarly `year, month, day, week, time, hour, minute, second` can be used to match other fields.
  - `fieldname__isnull` null test.
  - `fieldname__regex = regexpattern` regular expression test.
  - `iexact, istartswith, iendswith, icontains, iregex` are case insensitive versions of their corresponding lookups.
* Lookup parameters are combined with `AND` in SQL. as:

  `filter(name='Ali',sname__endswith='oğlu', sid__gt = '51231')` is executed as:
  
  `SELECT ... WHERE name='Ali' AND sname LIKE '%oğlu' AND sid__gt=51231;`
* `exclude(<lookup>)` Works like filter but negates the lookup.
* QuerySets can be chained as:

   `Student.filter(sid__ge = 10000).exclude(name__range = ('AA','IA')).filter(name__endswith = 'Z')`
* For conjunction (SQL `OR`) and complex expression in parenthesis, you need `Q` objects (`from django.db.models import Q)` Not in scope of this course.


In [12]:
print("--all--\n",Student.objects.all())
print("--get--\n",Student.objects.get(sid=423145))
print("--get--\n",Student.objects.get(name='Bugs'))
print("--filter--\n",Student.objects.filter(name__startswith='T'))
print("--filter--\n",Student.objects.filter(name__contains='ty'))
print("--filter-excl-\n",Student.objects.filter(name__contains='ty').exclude(sid__startswith = '4') )


--all--
 <QuerySet [<Student: 211112 Ali Cin>, <Student: 44444 Onur Tolga Şehitoğlu>, <Student: 123415 Bugs Bunny>, <Student: 423122 Daffy Duck>, <Student: 314213 Sylvester Cat>, <Student: 423145 Tweety Tweets>, <Student: 221412 Tazmanian Devil>, <Student: 23523523 weqwe qweqwe>, <Student: 324243 asdasd  lczxc zxzxc>, <Student: 55727 Onur Tolga Şehitoğlu>]>
--get--
 423145 Tweety Tweets
--get--
 123415 Bugs Bunny
--filter--
 <QuerySet [<Student: 423145 Tweety Tweets>, <Student: 221412 Tazmanian Devil>]>
--filter--
 <QuerySet [<Student: 423145 Tweety Tweets>]>
--filter-excl-
 <QuerySet []>


## Updating Data

* Single objects can be updated by fetching, updating field and saving as:
  ```python
   s=Student.objects.get(sid=55727)
   s.name = 'Onur'
   s.save()
  ```
* Also `update()` method can be used in query sets:
  ```python  
   s.Student.objects.filter(sid=55727).update(name='Onur')
   s.Student.objects.filter(name__gt = 'TT').update(count=0)
  ```
* `delete()` method on an object or query set deletes the object or objects in query set.



## Model Class Relations

* **One to Many** relation maps an object field into a set of other objects. For example a Manifacturer has multiple Car models, a Bus has multiple Passengers, a Department has multiple Students. But inverse does not hold, a Car has only one Manifacturer as brand, a Passenger can be on a single Bus at a time. A Student can register one major Department.
* **Many to Many** relation maps many objects into more than one objects. For example a Book can be written by more than one Authors and each Author can write more than one Book. A Student can register to many Courses and each Course has many Students. 
* **One to One** relation maps an object to another. For example Metadata of a file object is related to its Content, EmailBody is related to EmailHeader. This relation can be used as logical partitioning of the object.

Django provide this relations and creates required tables, fields, indexes automatically. Also provides SQL supported constraints so that when an object is deleted, the related object field is automatically set, object is deleted or error raised to enforce deletion of referring object first.

* `fieldname=ForeignKey('ClassName')` in a model creates a **One to Many** relation between the `ClassName` and the current class. 

  `Car.producedby=ForeignKey('Manifacturer')`, 
  `Passenger.passat=ForeignKey('Bus')`, 
  `Student.dept=ForeignKey('Department')`.
* `fieldname=ManytoManyField('ClassName')` creates a **Many to Many** relation.

  `Book.by=ManyToManyField('Author')`, `Student.registered=ManytoManyField('Course')`.
* `fieldname=OnetoOneField('ClassName')` creates a **One to One** relation.

The relation among referring class to referred class is through the defined field (`producedby, dept, by, registered`. In the referred class the inverse relation is defined by `classname_set`.
As in `Manifacturer`: `car_set`, in `Author`: `book_set`, in `Course`: `student_set` (class name is lowercased)

If there are more than one such relations or you need to give a more readable name, you can use `related_name` in the field definition.

Let us improve our model:
```python
class Department(models.Model):
	did = models.CharField(max_length=10, primary_key = True)
	name = models.CharField(max_length=30)

class Course(models.Model):
	cid = models.CharField(max_length=10, primary_key = True)
	name = models.CharField(max_length=100)
    # this is tricky, it creates  many to many relation to between Course and Course
    # symmetrical=True automatically makes relation bidirectional as in friend-friend,
    # if A is friend of B, B is friend of A. prerequisite relations is not like that
	prereq = models.ManyToManyField('self', symmetrical=False, blank=True)
	def __str__(self):
		return ':'.join([str(self.cid),str(self.name)])
	

class Student(models.Model):
	sid = models.CharField(max_length=10, primary_key = True)
	name = models.CharField(max_length=30)
	surname = models.CharField(max_length=30)
    # many to one from Department
	department = models.ForeignKey(Department,blank=True, null=True)
    # many to many with courses
	took = models.ManyToManyField(Course,related_name = 'taken')
    # many to many with courses
	registered = models.ManyToManyField(Course,related_name = 'enrolled')

	def __str__(self):
		return ' '.join([str(self.sid),str(self.name),str(self.surname)])
```

You need to call `manage.py makemigrations` afterwards.

This will create the following SQL tables (without indexes):
```SQL
CREATE TABLE "student_course" (
    "cid" varchar(10) NOT NULL PRIMARY KEY, 
    "name" varchar(100) NOT NULL);
    
CREATE TABLE "student_course_prereq" (
    "id" integer NOT NULL PRIMARY KEY AUTOINCREMENT,          
    "from_course_id" varchar(10) NOT NULL REFERENCES "student_course" ("cid"), 
    "to_course_id" varchar(10) NOT NULL REFERENCES "student_course" ("cid"));

CREATE TABLE "student_department" (
    "did" varchar(10) NOT NULL PRIMARY KEY, 
    "name" varchar(30) NOT NULL);

CREATE TABLE "student_student" (
    "sid" varchar(10) NOT NULL PRIMARY KEY, 
    "name" varchar(30) NOT NULL, 
    "surname" varchar(30) NOT NULL, 
    "department_id" varchar(10) NULL REFERENCES "student_department" ("did"));

CREATE TABLE "student_student_registered" (
    "id" integer NOT NULL PRIMARY KEY AUTOINCREMENT, 
    "student_id" varchar(10) NOT NULL REFERENCES "student_student" ("sid"), 
    "course_id" varchar(10) NOT NULL REFERENCES "student_course" ("cid"));
                                           
CREATE TABLE "student_student_took" (
    "id" integer NOT NULL PRIMARY KEY AUTOINCREMENT, 
    "student_id" varchar(10) NOT NULL REFERENCES "student_student" ("sid"), 
    "course_id" varchar(10) NOT NULL REFERENCES "student_course" ("cid"));                                           
                                           
```

In [13]:
for cid,cname in [('ceng100','Computer Engineering Orientation.'), ('ceng111','Introduction to Computer Engineering Concepts.'), 
('ceng140','C Programming.'), ('ceng213','Data Structures.'), ('ceng223','Discrete Computational Structures.'),
('ceng232','Logic Design.'), ('ceng242','Programming Language Concepts.'), ('ceng280','Formal Languages And Abstract Machines.'),
('ceng300','Summer Practice - I.'), ('ceng315','Algorithms.'), ('ceng331','Computer Organization.'),
('ceng334','Introduction to Operating Systems.'), ('ceng336','Int. to Embedded Systems Development.'), ('ceng350','Software Engineering.'),
('ceng351','Data Management And File Structures.'), ('ceng378','Computer Graphics - I.'), ('ceng384','Signals and Systems for Computer Engineers.'),
('ceng400','Summer Practice - II.'), ('ceng435','Data Communications and Networking.'), ('ceng436','Data Communications and Computer Networking.'),
('ceng477','Introduction to Computer Graphics.'), ('ceng491','Computer Engineering Design I.'), ('ceng492','Computer Engineering Design II.')]:
    Course.objects.create(cid=cid, name=cname)
for did,dname in [('572','Aerospace Engineering'), ('563','Chemical Engineering'), ('562','Civil Engineering'),
('571','Computer Engineering'), ('567','Electrical and Electronics Engineering'), ('560','Environmental Engineering'),
('573','Food Engineering'), ('564','Geological Engineering'), ('568','Industrial Engineering'),
('569','Mechanical Engineering'), ('570','Metallurgical and Materials Engineering'), ('565','Mining Engineering')]:
    Department.objects.create(did=did,name=dname)

IntegrityError: UNIQUE constraint failed: student_course.cid

In [14]:
print(Course.objects.all())
print(Department.objects.all())
print(Student.objects.all())

<QuerySet [<Course: ceng100:Computer Engineering Orientation.>, <Course: ceng111:Introduction to Computer Engineering Concepts.>, <Course: ceng140:C Programming.>, <Course: ceng213:Data Structures.>, <Course: ceng223:Discrete Computational Structures.>, <Course: ceng232:Logic Design.>, <Course: ceng242:Programming Language Concepts.>, <Course: ceng280:Formal Languages And Abstract Machines.>, <Course: ceng300:Summer Practice - I.>, <Course: ceng315:Algorithms.>, <Course: ceng331:Computer Organization.>, <Course: ceng334:Introduction to Operating Systems.>, <Course: ceng336:Int. to Embedded Systems Development.>, <Course: ceng350:Software Engineering.>, <Course: ceng351:Data Management And File Structures.>, <Course: ceng378:Computer Graphics - I.>, <Course: ceng384:Signals and Systems for Computer Engineers.>, <Course: ceng400:Summer Practice - II.>, <Course: ceng435:Data Communications and Networking.>, <Course: ceng436:Data Communications and Computer Networking.>, '...(remaining ele

In [7]:
s = Student.objects.get(sid='55727')
print(s)

55727 Onur Tolga Şehitoğlu


In [11]:
# setting a foreign key field
s.department = Department.objects.get(did='571')
s.save()

In [ ]:
# student_set is defined in Department to get all Students
d=Department(did='571')
# it is a set object that can be queried
d.student_set.all()


In [ ]:
# setting in inverse direction is also possible
d.student_set.add(Student.objects.get(sid='211112'))
d.student_set.all()

In [14]:
# ManytoMany fields return a set when inspected
print(repr(s.took))
# you canget queries on the sets
print(s.took.all())
s.took.add(Course.objects.get(cid='ceng350'))
s.took.add(Course.objects.get(cid='ceng315'))


<django.db.models.fields.related_descriptors.create_forward_many_to_many_manager.<locals>.ManyRelatedManager object at 0x7f532a56b908>
<QuerySet [<Course: ceng315:Algorithms.>, <Course: ceng350:Software Engineering.>, <Course: ceng491:Computer Engineering Design I.>]>


In [15]:
s.took.all()

<QuerySet [<Course: ceng315:Algorithms.>, <Course: ceng350:Software Engineering.>, <Course: ceng491:Computer Engineering Design I.>]>

In [16]:
c=Course.objects.get(cid='ceng350')
c.taken.all()

<QuerySet [<Student: 123415 Bugs Bunny>, <Student: 55727 Onur Tolga Şehitoğlu>, <Student: 423145 Tweety Tweets>]>

In [18]:
c.taken.add(Student.objects.get(name='Tweety'))

In [19]:
Student.objects.get(name='Tweety').took.all()

<QuerySet [<Course: ceng350:Software Engineering.>]>

In [20]:
qs = Student.objects.all()

In [21]:
for st in qs.order_by('surname'):
    print('\n===\n',st.name, st.surname)
    print('took:')
    for c in st.took.all():
        print('  ->', c.cid, c.name)
    print('registered:')
    for c in st.registered.all():
        print('  ->', c.cid, c.name)

for c in Course.objects.all().order_by('cid'):
    print('\n===\n', c.cid, c.name)
    for s in c.taken.all():
        print('  ->', s.sid, s.name, s.surname)
    for s in c.enrolled.all():
        print('  ->', s.sid, s.name, s.surname)
        
Student.objects.exclude(name__range=('AA','MM')).filter(sid__lt = '4')


===
 Bugs Bunny
took:
  -> ceng350 Software Engineering.
registered:
  -> ceng223 Discrete Computational Structures.
  -> ceng351 Data Management And File Structures.

===
 Sylvester Cat
took:
  -> ceng232 Logic Design.
registered:
  -> ceng100 Computer Engineering Orientation.
  -> ceng111 Introduction to Computer Engineering Concepts.
  -> ceng140 C Programming.
  -> ceng223 Discrete Computational Structures.
  -> ceng378 Computer Graphics - I.
  -> ceng384 Signals and Systems for Computer Engineers.
  -> ceng436 Data Communications and Computer Networking.

===
 Ali Cin
took:
  -> ceng140 C Programming.
registered:
  -> ceng111 Introduction to Computer Engineering Concepts.
  -> ceng232 Logic Design.

===
 Tazmanian Devil
took:
registered:
  -> ceng223 Discrete Computational Structures.

===
 Daffy Duck
took:
registered:

===
 Hara Gürele
took:
registered:

===
 Tweety Tweets
took:
  -> ceng350 Software Engineering.
registered:
  -> ceng100 Computer Engineering Orientation.

===
 H

<QuerySet [<Student: 221412 Tazmanian Devil>, <Student: 23523523 weqwe qweqwe>, <Student: 314213 Sylvester Cat>, <Student: 324243 asdasd  lczxc zxzxc>]>

In [22]:
Student.objects.all().values('sid','name')

<QuerySet [{'name': 'Ali', 'sid': '211112'}, {'name': 'Onur Tolga', 'sid': '44444'}, {'name': 'Bugs', 'sid': '123415'}, {'name': 'Daffy', 'sid': '423122'}, {'name': 'Sylvester', 'sid': '314213'}, {'name': 'Tweety', 'sid': '423145'}, {'name': 'Tazmanian', 'sid': '221412'}, {'name': 'Onur Tolga', 'sid': '55727'}, {'name': 'Hello', 'sid': '673452'}, {'name': 'weqwe', 'sid': '23523523'}, {'name': 'asdasd ', 'sid': '324243'}, {'name': 'Hara', 'sid': '523431'}]>

In [23]:
Student.objects.all()

<QuerySet [<Student: 211112 Ali Cin>, <Student: 44444 Onur Tolga Şehitoğlu>, <Student: 123415 Bugs Bunny>, <Student: 423122 Daffy Duck>, <Student: 314213 Sylvester Cat>, <Student: 423145 Tweety Tweets>, <Student: 221412 Tazmanian Devil>, <Student: 55727 Onur Tolga Şehitoğlu>, <Student: 673452 Hello World>, <Student: 23523523 weqwe qweqwe>, <Student: 324243 asdasd  lczxc zxzxc>, <Student: 523431 Hara Gürele>]>

In [24]:
Student.objects.filter(sid__gt = '5').delete()

(11,
 {'student.Student': 3,
  'student.Student_registered': 5,
  'student.Student_took': 3})

In [25]:
d = Department.objects.get(did='571')
d.student_set.all()

<QuerySet [<Student: 123415 Bugs Bunny>]>

In [26]:
d.student_set.add(Student.objects.get(name='Bugs'))        

In [27]:
d.student_set.all()

<QuerySet [<Student: 123415 Bugs Bunny>]>

## Meta Options
Extra constraints and behaviour of Models can be changed through a ```Meta``` subclass definition.

```Python
class Student(models.Model):
    sid = models.CharField(max_length=10, primary_key = True)
    name = models.CharField(max_length=30)
    surname = models.CharField(max_length=30)
    class Meta:
        db_table = 'metu_student'    # use a nonstandart DB table name
        ordering = ['surname','name']  # query results ordered by fields other than id
        indexes = [models.Index(fields=['surname','name']) ]  #multifield secondary indexes 
```

What is left?:
* advanced queries by using django.db.Q
* advanced update by django.db.F, i.e. increment a value without get()-update-save()
* aggrate functions django.db. Count, Avg, Sum, ...
* group by functionality through annote() and aggragate functions
* write your custom field lookups
* write your custom aggrate functions

In [19]:
from django.db.models import Q

In [20]:
# or query or any logical combinator with & and | 
Student.objects.filter(Q(name__gt='T') | Q(sid__lt='4000'))

<QuerySet [<Student: 211112 Ali Cin>, <Student: 123415 Bugs Bunny>, <Student: 314213 Sylvester Cat>, <Student: 423145 Tweety Tweets>, <Student: 221412 Tazmanian Devil>, <Student: 23523523 weqwe qweqwe>, <Student: 324243 asdasd  lczxc zxzxc>]>

In [36]:
from django.db.models import Avg,Max,Count,Min
allstd=Student.objects.all()
print("Count:",allstd.count())
print("Average:",allstd.aggregate(Avg('sid')))
print("Aggregate:", allstd.aggregate(Max('sid')), allstd.aggregate(Min('sid')))
print("===\ntook and registered counts per student:")
for v in Student.objects.values('sid','name','surname').annotate(Count('took'),Count('registered')):
    print("{sid:10s} {name:15s} {surname:15s} {took__count:4d} {registered__count:4d}".format(**v))
print('====\n',"enrolled per course")
for c in Course.objects.values('cid','name').annotate(Count('enrolled')):
    print(c)

Count: 10
Average: {'sid__avg': 2566435.6}
Aggregate: {'sid__max': '55727'} {'sid__min': '123415'}
===
took and registered counts per student:
123415     Bugs            Bunny              4    4
211112     Ali             Cin                2    2
221412     Tazmanian       Devil              0    1
23523523   weqwe           qweqwe             0    0
314213     Sylvester       Cat                7    7
324243     asdasd          lczxc zxzxc        0    0
423122     Daffy           Duck               0    0
423145     Tweety          Tweets             1    1
44444      Onur Tolga      Şehitoğlu          6    6
55727      Onur Tolga      Şehitoğlu          2    0
====
 enrolled per course
{'cid': 'ceng100', 'name': 'Computer Engineering Orientation.', 'enrolled__count': 2}
{'cid': 'ceng111', 'name': 'Introduction to Computer Engineering Concepts.', 'enrolled__count': 2}
{'cid': 'ceng140', 'name': 'C Programming.', 'enrolled__count': 1}
{'cid': 'ceng213', 'name': 'Data Structures.', 'e